<a href="https://colab.research.google.com/github/biwa7636/blog/blob/main/visualize_llm_size.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
%pip install -q transformers

In [68]:
import pandas as pd
import plotly.graph_objects as go
from transformers import AutoModelForCausalLM, BertModel

ckpt = 'bert-base-uncased'
model = BertModel.from_pretrained(ckpt)

In [69]:
labels = []
parents = []
values = []
texts = []

def traverse(module, parent=''):

    parents.append(parent)
    texts.append(module.__repr__().replace("\n", "<br>"))

    cur_num = sum(p.numel() for p in module.parameters())
    values.append(cur_num)

    cur_label = parent + '/' + module._get_name()
    # confirm cur_label is unique, otherwise, rename
    c = 1
    while 1:
        if cur_label not in labels:
            break
        elif cur_label + str(c) not in labels:
            cur_label += str(c)
            break
        else:
            c += 1
    labels.append(cur_label)

    for name, child in module.named_children():
        traverse(child, parent=cur_label)


traverse(model, parent='')

In [70]:
fig = go.Figure(go.Treemap(
    labels = labels,
    parents = parents,
    values = values,
    text = texts,
    hoverlabel=dict(bgcolor='white')
))

fig.update_traces(root_color='lightgrey')
fig.show()

In [74]:
fig = go.Figure(go.Sunburst(
    labels = labels,
    parents = parents,
    values = values,
))

fig.show()